<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AGENT_DEMO_Neoantigen_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import sys
# Import the necessary components for the Google GenAI SDK
from google import genai
from google.genai import types

# IMPORTANT: This import is required for using the userdata.get() function in Colab
try:
    from google.colab import userdata
except ImportError:
    print("Warning: 'google.colab' not found. This code is intended for Google Colab.")

# --- 1. Client Initialization (STRICTLY using user-specified setup) ---

# --- START User-Specified Initialization Block ---

try:
    # 1. Retrieve the secret API key from Colab Secrets
    # This line uses the exact variable name 'GEMINI' as stored in your Colab Secrets
    GOOGLE_API_KEY = userdata.get('GEMINI')

    if not GOOGLE_API_KEY:
        # 2. Check for the key and raise the required error if not found
        raise ValueError("Error: 'GEMINI' API key not found in Colab Secrets. Please check the Secrets tab.")

    # 3. Initialize the client with the retrieved key
    client = genai.Client(api_key=GOOGLE_API_KEY)
    GEMINI_MODEL_ID = 'gemini-3-pro-preview'

except Exception as e:
    print(f"❌ Initialization Error: {e}")
    sys.exit(1)

# --- END User-Specified Initialization Block ---

print(f"✅ Gemini Client Initialized successfully using model: {GEMINI_MODEL_ID}")


# --- 2. Define the Structured Output Schema ---
# This is the strict JSON contract for the Neoantigen Selection Agent.

ORDA_REQUEST_SCHEMA = types.Schema(
    type=types.Type.OBJECT,
    description="Manifest for initiating the Neoantigen Selection Agent based on patient genomics data.",
    properties={
        'workflow_step': types.Schema(
            type=types.Type.STRING,
            description='The next agent in the sequence, must be "Neoantigen_Selection".'
        ),
        'patient_id': types.Schema(
            type=types.Type.STRING,
            description='Unique identifier for the patient.'
        ),
        'priority_targets': types.Schema(
            type=types.Type.ARRAY,
            items=types.Schema(type=types.Type.STRING),
            description='The 3-5 most critical, highly expressed mutation targets to prioritize.'
        ),
        'max_epitopes': types.Schema(
            type=types.Type.INTEGER,
            description='The maximum number of final epitopes allowed for vaccine synthesis (e.g., 25).'
        ),
        'optimization_flags': types.Schema(
            type=types.Type.OBJECT,
            properties={
                'Stability_Score_Min': types.Schema(type=types.Type.NUMBER),
                'Translation_Efficiency_Target': types.Schema(type=types.Type.NUMBER)
            }
        )
    },
    required=['workflow_step', 'patient_id', 'priority_targets', 'max_epitopes', 'optimization_flags']
)

# --- 3. Input Data and Prompt Construction ---

patient_data_summary = {
    "patient_id": "ORDA-PC-JM-001",
    "cancer_type": "Pancreatic Adenocarcinoma (Stage III)",
    "hla_type": "HLA-A*02:01",
    "key_mutations": ["KRAS G12D", "TP53 R248W", "SMAD4 D493V", "CDKN2A D51A"],
    "neoantigen_candidates_count": 215,
    "synthesis_limit": 25
}

prompt = f"""
**ROLE:** You are the **ORDA Workflow Orchestrator**. Act as an agent managing the pipeline.
**TASK:** Review the patient data summary and generate the structured JSON request to initiate the 'Neoantigen Selection Agent'.

PATIENT DATA SUMMARY (from Genomics Agent):
{json.dumps(patient_data_summary, indent=4)}

**INSTRUCTIONS:**
1. Set the 'workflow_step' to 'Neoantigen_Selection'.
2. Select the three most critical key mutations for the 'priority_targets' array: KRAS G12D, TP53 R248W, and SMAD4 D493V.
3. Set 'max_epitopes' to the synthesis limit (25).
4. Set the optimization flag values based on high-performance mRNA vaccine standards:
    - Stability_Score_Min: 0.92
    - Translation_Efficiency_Target: 0.90

**Output ONLY the JSON object that conforms precisely to the provided schema.**
"""

# --- 4. Gemini API Call with Structured Output Configuration ---

print("\n🚀 Sending request to Gemini 3 Pro (Agentic Orchestration using structured output)...")

try:
    # Configure the generation request to enforce the JSON structure defined above
    config = types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=ORDA_REQUEST_SCHEMA,
    )

    response = client.models.generate_content(
        model=GEMINI_MODEL_ID,
        contents=prompt,
        config=config,
    )

    # --- 5. Output Processing and Validation ---

    response_json = json.loads(response.text)

    print("\n✅ ORDA Agent Manifest Generated (Valid JSON for Next Agent):")
    print("---------------------------------------------------------")
    print(json.dumps(response_json, indent=4))
    print("---------------------------------------------------------")

    next_agent = response_json.get('workflow_step')
    print(f"\n➡️ SUCCESS: The Workflow Orchestrator successfully routed the manifest to the '{next_agent}' Agent.")


except Exception as e:
    # This error handling now relies on the valid key from the initialization step
    print(f"\n❌ A critical error occurred during the API call: {e}")

✅ Gemini Client Initialized successfully using model: gemini-3-pro-preview

🚀 Sending request to Gemini 3 Pro (Agentic Orchestration using structured output)...

✅ ORDA Agent Manifest Generated (Valid JSON for Next Agent):
---------------------------------------------------------
{
    "workflow_step": "Neoantigen_Selection",
    "patient_id": "ORDA-PC-JM-001",
    "priority_targets": [
        "KRAS G12D",
        "TP53 R248W",
        "SMAD4 D493V"
    ],
    "max_epitopes": 25,
    "optimization_flags": {
        "Stability_Score_Min": 0.92,
        "Translation_Efficiency_Target": 0.9
    }
}
---------------------------------------------------------

➡️ SUCCESS: The Workflow Orchestrator successfully routed the manifest to the 'Neoantigen_Selection' Agent.
